# Task Bonus

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

In [ ]:
def load_mnist_data(batch_size=10):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

In [ ]:
train_loader, test_loader = load_mnist_data(batch_size=10)

100%|██████████| 9912422/9912422 [00:00<00:00, 292077075.02it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 73683512.06it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 156926128.22it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6806191.06it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## 1. Compare 3 different configurations while your model is wider/deeper. Show and explain the performance result.

In [ ]:
 def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images.view(-1, 28 * 28))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        average_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {average_loss:.4f}')

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images.view(-1, 28 * 28))
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.numpy())
            all_preds.extend(preds.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return accuracy

In [ ]:
class WideModel(nn.Module):
    def __init__(self):
        super(WideModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

wide_model = WideModel()
optimizer = optim.SGD(wide_model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()
num_epochs = 10
train(wide_model, train_loader, criterion, optimizer, num_epochs)
accuracy1 = evaluate_model(wide_model, test_loader)
print(f'Accuracy (Wide Model): {accuracy1:.4f}')

Epoch [1/10] Loss: 0.3617
Epoch [2/10] Loss: 0.2043
Epoch [3/10] Loss: 0.1704
Epoch [4/10] Loss: 0.1539
Epoch [5/10] Loss: 0.1425
Epoch [6/10] Loss: 0.1229
Epoch [7/10] Loss: 0.1226
Epoch [8/10] Loss: 0.1127
Epoch [9/10] Loss: 0.1057
Epoch [10/10] Loss: 0.0997
Accuracy (Wide Model): 0.9617


In [ ]:
class DeepModel(nn.Module):
    def __init__(self):
        super(DeepModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

deep_model = DeepModel()
optimizer = optim.SGD(deep_model.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
train(deep_model, train_loader, criterion, optimizer, num_epochs)
accuracy2 = evaluate_model(deep_model, test_loader)
print(f'Accuracy (Deep Model): {accuracy2:.4f}')

Epoch [1/10] Loss: 0.4470
Epoch [2/10] Loss: 0.2198
Epoch [3/10] Loss: 0.1736
Epoch [4/10] Loss: 0.1512
Epoch [5/10] Loss: 0.1318
Epoch [6/10] Loss: 0.1202
Epoch [7/10] Loss: 0.1163
Epoch [8/10] Loss: 0.1070
Epoch [9/10] Loss: 0.0968
Epoch [10/10] Loss: 0.0976
Accuracy (Deep Model): 0.9635


In [ ]:
class OriginalModel(nn.Module):
    def __init__(self):
        super(OriginalModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

original_model = OriginalModel()
optimizer = optim.SGD(original_model.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
train(original_model, train_loader, criterion, optimizer, num_epochs)
accuracy3 = evaluate_model(original_model, test_loader)
print(f'Accuracy (Original Model): {accuracy3:.4f}')

Epoch [1/10] Loss: 0.4165
Epoch [2/10] Loss: 0.2661
Epoch [3/10] Loss: 0.2314
Epoch [4/10] Loss: 0.2027
Epoch [5/10] Loss: 0.1887
Epoch [6/10] Loss: 0.1726
Epoch [7/10] Loss: 0.1641
Epoch [8/10] Loss: 0.1551
Epoch [9/10] Loss: 0.1522
Epoch [10/10] Loss: 0.1464
Accuracy (Original Model): 0.9475


Berdasarkan hasil perhitungan untuk mendapatkan nilai akurasi dari 3 model tersebut, yaitu:

1. WideModel = Accuracy (Wide Model): 0.9617
2. DeepModel = Accuracy (Deep Model): 0.9635
3. OriginalModel = Accuracy (Original Model): 0.9475

Maka, dapat disimpulkan bahwa akurasi performa dari ketiga model di atas. Model yang memiliki akurasi tertinggi pada data uji adalah DeepModel = Accuracy (Deep Model): 0.9635 merupakan model pilihan terbaik.

## 2. Compare 3 configurations for different Loss Function. Show and explain your performance result

In [ ]:
train_loader, test_loader = load_mnist_data(batch_size=10)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
model1 = NeuralNetwork()
criterion1 = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
train(model1, train_loader, criterion1, optimizer, num_epochs)
accuracy1 = evaluate_model(model1, test_loader)
print(f'Accuracy (CrossEntropyLoss): {accuracy1:.4f}')

Epoch [1/10] Loss: 0.4087
Epoch [2/10] Loss: 0.2625
Epoch [3/10] Loss: 0.2230
Epoch [4/10] Loss: 0.2056
Epoch [5/10] Loss: 0.1842
Epoch [6/10] Loss: 0.1717
Epoch [7/10] Loss: 0.1619
Epoch [8/10] Loss: 0.1578
Epoch [9/10] Loss: 0.1562
Epoch [10/10] Loss: 0.1467
Accuracy (CrossEntropyLoss): 0.9545


In [ ]:
model2 = NeuralNetwork()
criterion2 = nn.NLLLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
train(model2, train_loader, criterion2, optimizer, num_epochs)
accuracy2 = evaluate_model(model2, test_loader)
print(f'Accuracy (NLLLoss): {accuracy2:.4f}')

Epoch [1/10] Loss: nan
Epoch [2/10] Loss: nan
Epoch [3/10] Loss: nan
Epoch [4/10] Loss: nan
Epoch [5/10] Loss: nan
Epoch [6/10] Loss: nan
Epoch [7/10] Loss: nan
Epoch [8/10] Loss: nan
Epoch [9/10] Loss: nan
Epoch [10/10] Loss: nan
Accuracy (NLLLoss): 0.0980


In [ ]:
model3 = NeuralNetwork()
criterion3 = nn.MSELoss()
optimizer = optim.SGD(model3.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
for epoch in range(num_epochs):
        model3.train()
        running_loss = 0.0
        for data in train_loader:
            x, y = data
            optimizer.zero_grad()
            outputs = model3(x.view(-1, 28*28))

            # Konversi label target menjadi Float
            y = y.float()

            loss = criterion3(outputs, y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        average_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}] Loss: {average_loss:.4f}')

accuracy3 = evaluate_model(model3, test_loader)
print(f'Accuracy (MSELoss): {accuracy3:.4f}')

Epoch [1/10] Loss: 8.5634
Epoch [2/10] Loss: 8.4311
Epoch [3/10] Loss: 8.4317
Epoch [4/10] Loss: 8.4295
Epoch [5/10] Loss: 8.4325
Epoch [6/10] Loss: 8.4304
Epoch [7/10] Loss: 8.4362
Epoch [8/10] Loss: 8.4336
Epoch [9/10] Loss: 8.4296
Epoch [10/10] Loss: 8.4290
Accuracy (MSELoss): 0.1028


Berdasarkan hasil perhitungan untuk mendapatkan nilai akurasi dari 3 model tersebut, yaitu:

1. CrossEntropyLoss = Accuracy (CrossEntropyLoss): 0.9545
2. NLLLoss = Accuracy (NLLLoss): 0.0980
3. MSELoss = Accuracy (MSELoss): 0.1028

Maka, dapat disimpulkan bahwa akurasi performa dari ketiga model di atas. Model yang memiliki akurasi tertinggi pada data uji adalah CrossEntropyLoss = Accuracy (CrossEntropyLoss): 0.9545 merupakan model pilihan terbaik.

## 3. Compare 3 configurations for the activation function. Show and explain your performance result

In [ ]:
train_loader, test_loader = load_mnist_data(batch_size=10)

In [ ]:
class ReLUModel(nn.Module):
    def __init__(self):
        super(ReLUModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model1 = ReLUModel()
optimizer = optim.SGD(model1.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
train(model1, train_loader, criterion1, optimizer, num_epochs)
accuracy1 = evaluate_model(model1, test_loader)
print(f'Accuracy (ReLU): {accuracy1:.4f}')

Epoch 1, Loss: 0.41878489256236207
Epoch 2, Loss: 0.27148670624190585
Epoch 3, Loss: 0.23239608119466568
Epoch 4, Loss: 0.2095702025466256
Epoch 5, Loss: 0.1935319307468347
Epoch 6, Loss: 0.18925841233344445
Epoch 7, Loss: 0.1826826002076441
Epoch 8, Loss: 0.17547760468849075
Epoch 9, Loss: 0.16851109756781033
Epoch 10, Loss: 0.16587752328297634
Accuracy (ReLU): 0.9514


In [ ]:
class SigmoidModel(nn.Module):
    def __init__(self):
        super(SigmoidModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x
model2 = SigmoidModel()
optimizer = optim.SGD(model2.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
train(model2, train_loader, criterion1, optimizer, num_epochs)
accuracy2 = evaluate_model(model2, test_loader)
print(f'Accuracy (Sigmoid): {accuracy2:.4f}')

Epoch 1, Loss: 0.34865920633248365
Epoch 2, Loss: 0.17011614429527738
Epoch 3, Loss: 0.12446483051976732
Epoch 4, Loss: 0.09867595609852772
Epoch 5, Loss: 0.08204613654753
Epoch 6, Loss: 0.07015426817190504
Epoch 7, Loss: 0.06125538524226916
Epoch 8, Loss: 0.05438644877596865
Epoch 9, Loss: 0.048017448095786675
Epoch 10, Loss: 0.042742325718501284
Accuracy (Sigmoid): 0.9774


In [ ]:
class TanhModel(nn.Module):
    def __init__(self):
        super(TanhModel, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x
model3 = TanhModel()
optimizer = optim.SGD(model3.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
train(model3, train_loader, criterion1, optimizer, num_epochs)
accuracy3 = evaluate_model(model3, test_loader)
print(f'Accuracy (Tanh): {accuracy3:.4f}')

Epoch 1, Loss: 0.4296437833693344
Epoch 2, Loss: 0.4044272551409279
Epoch 3, Loss: 0.3928960522466805
Epoch 4, Loss: 0.3826399141418127
Epoch 5, Loss: 0.3670486559078951
Epoch 6, Loss: 0.33130472946110723
Epoch 7, Loss: 0.31381327990776237
Epoch 8, Loss: 0.3159878531287347
Epoch 9, Loss: 0.2965952137750962
Epoch 10, Loss: 0.28008833530296884
Accuracy (Tanh): 0.9266


Berdasarkan hasil perhitungan untuk mendapatkan nilai akurasi dari 3 model tersebut, yaitu:

1. ReLU = Accuracy (ReLU): 0.9514
2. Sigmoid = Accuracy (Sigmoid): 0.9774
3. Tanh = Accuracy (Tanh): 0.9266

Maka, dapat disimpulkan bahwa akurasi performa dari ketiga model di atas. Model yang memiliki akurasi tertinggi pada data uji adalah Sigmoid = Accuracy (Sigmoid): 0.9774 merupakan model pilihan terbaik.